In [1]:
from aux.aux_functions import *

In [2]:
import openai
from openai import OpenAI

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

from IPython.display import display

In [4]:
from dotenv import load_dotenv
import os

load_dotenv("/Users/fmuia/Dropbox/NLP/Llama-FineTuning/config.env")
open_ai_key = os.getenv("OPENAI_API_KEY")

In [5]:
# Specify the path to your LaTeX file
latex_file_path = 'Files/2310.20559/main.tex'

# Extract the LaTeX content
latex_content = extract_latex_content(latex_file_path)

# For demonstration, let's print the first 500 characters to check if it worked
#print(latex_content[:15000])

In [10]:
simplified_text = replace_latex_commands(latex_content)
#simplified_text

In [11]:
polished_text = clean_latex_content(simplified_text)
#polished_text

In [12]:
no_char_in_chunk = 2000

chunks = chunk_text(polished_text, no_char_in_chunk)
chunks = [chunk.replace("%", "") for chunk in chunks]
print("The number of chunks is: " + str(len(chunks)) + ".")
#chunks[:2]

The number of chunks is: 143.


In [14]:
# Define the number of Q&A pairs you want to generate for each chunk
num_questions = 3

In [15]:
system_prompt_template = "You are an expert in theoretical physics. I will provide a text, and I'd like you to generate {num_questions} questions a researcher-level colleague might ask about the topic of the text. Then, provide detailed answers to these questions as an expeert in the field. Format your responses as follows: 'Question 1: [question here] Answer 1: [answer here]' and so on for all {num_questions} questions."

user_prompt_template = "Based on the following text, please generate the questions and answers as instructed. Text: {chunk}"

display(system_prompt_template)
display(user_prompt_template)


"You are an expert in theoretical physics. I will provide a text, and I'd like you to generate {num_questions} questions a researcher-level colleague might ask about the topic of the text. Then, provide detailed answers to these questions as an expeert in the field. Format your responses as follows: 'Question 1: [question here] Answer 1: [answer here]' and so on for all {num_questions} questions."

'Based on the following text, please generate the questions and answers as instructed. Text: {chunk}'

In [16]:
client = OpenAI()

# Initialize a list to store chunk and response pairs
chunk_responses = []

# Assume chunks_test is a list of text chunks you want to generate Q&A for
for chunk in chunks:
    system_prompt, user_prompt = generate_prompt(chunk, num_questions, system_prompt_template, user_prompt_template)
    
    # Call the OpenAI API once per chunk
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[system_prompt, user_prompt]
    )
    
    # Append both the chunk and its response to the chunk_responses list
    chunk_responses.append({
        "chunk": chunk,
        "response": response.choices[-1].message.content if response.choices else "No response"
    })


In [17]:
# Initialize lists to hold the DataFrame data
chunk_list = []
question_list = []
answer_list = []
sub_index = []

# Process each chunk-response pair
for item in chunk_responses:
    chunk = item['chunk']
    response = item['response']
    
    # Split the response into Q&As
    qas = response.split('\n\n')  # Assuming two newlines separate each Q&A pair
    for i, qa in enumerate(qas):
        # Attempt to split the QA into question and answer
        parts = qa.split('\nAnswer ')
        if len(parts) == 2:
            question = parts[0].replace('Question {}: '.format(i+1), '').strip()
            answer = parts[1].strip()
            chunk_list.append(chunk)
            question_list.append(question)
            answer_list.append(answer)
            sub_index.append(i+1)

In [20]:
# Create a MultiIndex from the chunks and their corresponding sub_index
multi_index = pd.MultiIndex.from_arrays([chunk_list, sub_index], names=['Chunk', 'Q&A Index'])
answer_list = [re.sub(r'^\d+: ', '', answer) for answer in answer_list]

# Construct the DataFrame
df_multi = pd.DataFrame({'Question': question_list, 'Answer': answer_list}, index=multi_index)

# Display the DataFrame
df_multi

Question  \
Chunk                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Q&A Index                                                                                                                                                                                   
Liam McAllister$^{a}$ and Fernando Quevedo$^{b}$ 30   We give an overview of moduli stabilization in compactifications of string theory. We summarize current methods for construction and analysis of vacua with stabilized moduli, and we describe applications to cosmology and particle physics. This is a contribution to the Handbook on Quantum Gravity.  The fundamental physical laws that govern our Universe must describe gravity and quantum mechanics. To discover the laws of quantum gravity, we cannot entirely rely on terrestrial experiments, or even on cosmological observations: the energies of observable processes are far too low to give a complete picture, in contrast to the way that collider experiments eventually revealed the Standard Model of particle physics. We may hope for some guidance from experiment, but theorists will have to provide a framework. String theory is such a framework: it is a theory of quantum gravity through which we can take a constructive approach to exploring possible laws of quantum gravity in our Universe. The first obstacle is that the world we observe at low energies is four-dimensional, while the best-understood solutions of string theory are ten-dimensional. Kaluza-Klein theory \cite{Kaluza:1921tu,Klein:1926tv}, now more than a century old, provides a way to bridge this gap. If the extra dimensions correspond to a six-dimensional compact space that is smaller than the reach of any experimental probe then only three spatial dimensions will be seen. However, the size and shape of the extra dimensions are dynamical: they are parameterized by the expectation values of scalar fields known as moduli. Unless the moduli have large masses, they mediate long-range forces that are not observed in our world. Thus, a central problem of Kaluza-Klein theories is to provid

In [23]:
qa_df = pd.DataFrame({
    'Question': question_list,
    'Answer': answer_list
})

qa_df = qa_df.iloc[0:352]
qa_df

,Question,Answer
0,What are the main challenges in moduli stabilization in compactifications of string theory?,"The main challenges in moduli stabilization in compactifications of string theory include ensuring that the size and shape of extra dimensions are dynamically determined by the expectation values of scalar fields (moduli) and that the masses of these moduli are consistent with experimental observations. Additionally, ensuring that the moduli do not mediate long-range forces that contradict our observed world is crucial."
1,"How does string theory provide a framework for exploring laws of quantum gravity, and why is it necessary in our quest to understand the Universe's fundamental physical laws?","String theory provides a theoretical framework for exploring quantum gravity by unifying gravity with quantum mechanics. It is necessary because terrestrial experiments and cosmological observations do not provide high enough energies for a complete understanding of quantum gravity. With string theory, theorists can constructively explore possible laws of quantum gravity and address the discrepancies between the observed four-dimensional world and the underlying ten-dimensional nature of string theory."
2,"How does Kaluza-Klein theory address the issue of extra dimensions in the context of string theory, and what role do moduli play in this framework?","Kaluza-Klein theory helps bridge the gap between the observed four-dimensional world and the ten-dimensional nature of string theory by proposing that the extra dimensions are compactified on a six-dimensional space that is smaller than detectable by current experiments. The moduli, scalar fields in string theory, parameterize the size and shape of these extra dimensions. Moduli stabilization aims to dynamically fix the values of these fields to ensure the extra dimensions are consistent with our observed world and do not lead to long-range forces not seen in nature."
3,What is the central problem concerning Kaluza-Klein theories mentioned in the text and why is it crucial for connecting string theory to observations?,The central problem of Kaluza-Klein theories mentioned in the text is providing a dynamical explanation for the size of the extra dimensions and ensuring that the masses of the moduli are consistent with experimental observations. This is crucial for connecting string theory to observations because it directly impacts the predictions made by these theories and their compatibility with experimental results.
4,How do Ricci-flat six-manifolds play a role in providing vacuum solutions of string theory as discussed in the text?,"In the context of the text, the vacuum solutions of string theory are described by Ricci-flat six-manifolds. This means that for the ten-dimensional vacuum Einstein equations to be solved, the Ricci tensors corresponding to the internal compact space and the four-dimensional spacetime must be zero. Therefore, vacuum solutions of string theory are established by utilizing Ricci-flat six-manifolds."
...,...,...
347,How do weak and strong couplings in the context of moduli and scalar potentials impact the appearance of non-trivial vacua in string theory?,"In the context of moduli and scalar potentials, weak couplings at high energies play a significant role in the appearance of non-trivial vacua in string theory. The Dine-Seiberg problem highlights that precise control is challenging and typically guaranteed in regions of moduli space where the scalar potential is a runaway. Non-trivial vacua most naturally appear in regions of strong coupling, indicating the importance of understanding and controlling coupling strengths for stabilizing moduli."
348,"What are some strategic choices made in moduli stabilization scenarios like KKLT and LVS, and how do these choices lead to computable minima and controllable solutions?","In moduli stabilization scenarios such as KKLT (Kachru-Kallosh-Linde-Trivedi) and LVS (Large Volume Scenario), strategic choices

In [28]:
qa_df.to_csv("Datasets/2310.20559.csv")
qa_df.to_parquet("Datasets/2310.20559.parquet")